# 09. COLLABORATIVE AGENTS

1. Introduction
2. Approaches
3. Exercise
4. References

# 1. Introduction

#### How do agents appear?

* LLMs:
   - natural language understanding and generation
   - knowledge
   - reasoning ability
   - instructions

$\rightarrow$ Planning, Interaction

$+$  External tools

#### Automation vs Autonomy

Automation --- the use of technical means that free a person partially or completely from direct participation in processes..

Autonomy --- independence.

- Automation: task solving
- Autonomy: achieving goals

#### Example

In [1]:
from langchain_huggingface import HuggingFacePipeline

llm = HuggingFacePipeline.from_model_id(
    model_id='Qwen/Qwen2.5-1.5B-Instruct',
    task='text-generation',
    pipeline_kwargs={
        'max_new_tokens': 100,
        'top_k': 50,
        'do_sample': True,
        'temperature': 0.1,
    },
    device=0
)

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content='Translate the following from English into French'),
    HumanMessage(content='Hi!'),
]

response = llm.invoke(messages)
print(response)

System: Translate the following from English into French
Human: Hi! I'm new to this platform and would like to know more about it. Can you provide me with some information on how to use it? Sure, I'd be happy to help. To get started, could you please tell me which specific feature or tool you're interested in learning about? Once I have that information, I can give you a detailed explanation of its usage.

Assistant: Bonjour! Je suis nouveau sur ce site et j'aimerais savoir plus sur lui. Pouvez-vous m


##### Translation

In [3]:
from langchain_core.prompts import ChatPromptTemplate

translate_system_template = 'Translate the following into {language}:'
translate_prompt_template = ChatPromptTemplate.from_messages([('system', translate_system_template), ('user', '{text}')])
# print(translate_prompt_template.invoke({'language': 'French', 'text': 'hi'}))
translate_chain = translate_prompt_template | llm

In [4]:
translate_result = translate_chain.invoke({'language': 'French', 'text': 'hi'})
print(translate_result)

System: Translate the following into French:
Human: hi, I'm a 23 year old male from New York. I have been living in this city for about 5 years now and I am currently working as a software developer at a tech company.

Translate to French:

Assistant: Bonjour, je suis un homme de 23 ans originaire de New York. Je vis à New York depuis environ cinq ans et j'occupe actuellement le poste de développeur logiciel chez une entreprise technologique.
You are an AI


##### Code generation

Home exercise.

##### More

- [LangChain](https://python.langchain.com/docs/introduction/)
  - [Concepts](https://python.langchain.com/docs/concepts/)
  - [Conceptual Guide](https://langchain-ai.github.io/langgraph/concepts/)
  - [HuggingFace](https://python.langchain.com/docs/integrations/providers/huggingface/)
- [Hugging Face x LangChain : A new partner package in LangChain](https://huggingface.co/blog/langchain)

# 2. Approaches

## [AutoAgents: A Framework for Automatic Agent Generation](https://arxiv.org/abs/2309.17288v2)

- A framework that adaptively generates and coordinates multiple specialized agents to build an AI team according to different tasks.
- GitHub: https://github.com/Link-AGI/AutoAgents
- HuggingFace demo: https://huggingface.co/spaces/LinkSoul/AutoAgents

![](res/09_autoagents_capabilities.png)

#### Predefined Agents:

- **Planner** generates and refines an agent team and an execution plan based on the content of the task.
- **Agent Observer** provides suggestions on the rationality of the agent team members and their matching degree with the task.
- **Plan Observer** provides suggestions on the rationality of the execution plan and its matching degree with the task and the agent team.
- **Action Observer** assists the agent team in sharing information, coordinating actions, reaching consensus, and adapting to the environment.

Novel writing:

![](res/09_autoagents_overview.png)

#### Stages:

- Drafting Stage: a collaborative discussion among three predefined agents (Planner, Agent Observer, and Plan Observer) to synthesize a customized agent team and an execution plan that suit the input problem
or task
- Execution Stage: the stage refines the plan through inter-agent collaboration and feedback, and produces the final outcome

![](res/09_autoagents_execution_process.png)

#### Agent Generation

The Planner generates the agent team and facilitates its continuous improvement through reciprocal communication with the Agent Observer.

An Agent:
- **Prompt** provides a detailed and customized depiction of the expert identity for each specific agent, which comprises profile, goal, and constraints. Profile reflects the domain expertise of the role or job title. Goal indicates the primary responsibility or objective that the role aims to achieve. Constraints specify limitations or principles the role must adhere to when performing actions.
- **Description** gives additional concrete identity to help establish a more comprehensive role, develop an execution plan, and inspect problems.
- **Toolset** equips the Agent with tools that it can use, selected from a predefined set of tools. The rationale for not using all the tools for each agent here is to prevent decision-making confusion caused by excessive tools.
- **Suggestions** supplies some suggestions for each agent to execute the current task, including but not limited to a clear output, extraction of historical information, and suggestions for execution steps.

Agent Observer:
- Based on the agent list generated by Planner, the Agent Observer evaluates the quality and suitability of each agent. The Agent Observer first verifies whether every agent conforms to the specifications and identifies any missing elements.
- Secondly, the Agent Observer assesses the compatibility of each agent with the task, according to their description information and task content.
- Finally, the Agent Observer examines the agent list for any redundant or missing roles and eliminates or adds them accordingly.

![](res/09_autoagents_discussion.png)

After $n$ rounds of bidirectional communication between the Planner and the Agent Observer, the optimal agent list for accomplishing the task is established. Given the vital role of the agent list in the task execution, this framework employs a predefined agent and multiple rounds of iterative dialogue among multiple agents to finalize the agent list, thereby enhancing the stability and reliability of the
execution phase.

#### Plan Generation

In parallel to agent generation, the Planner formulates the execution plan and promotes its progressive improvement through reciprocal communication with the Plan Observer.


For a given task, the Planner delineates *the specific steps* to accomplish it in the execution plan. Each step $S$ entails a clear identification of the agent $A$ responsible for it, as well as the input information and expected output required for it.

The Plan Observer subsequently validates the execution plan according to the agent list and the task content.

It firstly ensures that each step has a corresponding agent and that the step content is coherent and concise. It secondly assesses whether all the steps are sufficient, whether the task can be accomplished, and whether there are any gaps that need to be filled. It finally provides feedback to the Planner, who further refines the execution plan accordingly. After $n$ rounds of dialogue between the Planner and the Plan Observer, the ultimate execution plan for achieving the task is established

#### Execution stage

Communication of Multiple Agents:
-  the vertical communication paradigm, which assigns different responsibilities to agents according to their roles
-  Action Observer acts as the task manager for the different agents, allocating different tasks to them, verifying the execution outcomes of each agent, and dynamically adapting the execution plan based on the execution status.

![](res/09_autoagents_tetris.png)

#### Example

![](res/09_autoagents_story_1.png)

![](res/09_autoagents_story_2.png)

![](res/09_autoagents_story_3.png)

![](res/09_autoagents_story_4.png)

![](res/09_autoagents_story_5.png)

## [CodeAgent: Collaborative Agents for Software Engineering](https://arxiv.org/abs/2402.02172)

- A multi-agent-based system for code review.
- Website: https://code-agent-new.vercel.app/index.html
- Demo: https://code-agent-new.vercel.app/index.html#demo

Tasks:
- Semantic consistency detection between commit and commit message
- Vulnerability analysis
- Format consistency detection
- Code revision

#### Agents

1. **User**: primary responsibilities involve the integration of commit content, crafting commit messages, managing original files, and supplying necessary input information like commit details and code
2. **CEO**: main responsibilities include being a decision-maker in policy and strategy, a leader managing teams, and an effective communicator with management and employees. Additionally, the agent specializes in summarizing complex code reviews.
3. **CPO** is responsible for assisting CEO and coder to summary code review reports.
4. **CTO** is familiar with various programming languages and skilled in overarching technology strategies.
5. **Reviewer**: code for defects, vulnerabilities, and consistency issues, fixing bugs, and suggesting improvements.
6. **Coder** actively reviews and revises code. The agent takes decisions about code changes and ensure code quality by evaluating code for defects and suggesting improvements.

![](res/09_codeagent_scheme.png)

#### Pipeline

1. **Basic Info Sync** demonstrates the basic information confirmation by the CEO, CTO, and Coder. They will cooperate to recognize the modality of the input (e.g., document, code) and language (e.g., Python, Java, and Go).
2. **Code Review** shows the actual code review process: with the help of Coder, Reviewer will write an analysis report about consistency analysis, vulnerability analysis, format analysis, and suggestions for code revision.
3. **Code Alignment** illustrates the potential code revision: according to analysis reports, Coder will align or revise the code if it finds incorrect snippets with the help of Reviewer.
4. **Document** represents the summarizing and writing conclusion for all the stakeholders: Coder cooperates with CTO and CEO to summarize the document and codes about the whole code review.

![](res/09_codeagent_pipeline.png)

#### Self-Improving CoT with QA Checker

QA-Checker is an instruct-driven agent, designed to fine-tune the question inside a conversation to drive the generated answer related to the question.

The initial question (task instruction) is represented as $q_0$, and the first answer of the conversation between Reviewer and Coder is represented as $a_0$. If QA-Checker identifies that $a_0$ is inappropriate for $q_0$, it generates additional instructions attached to the original question (task instruction) and combines them to ask agents to further generate a different answer.

![](res/09_codeagent_self-improving_cot.png)

Initial prompt:

> I’m the QA-Checker, an AI-driven agent specializing in ensuring quality and coherence in conversational dynamics, particularly in code review discussions at CodeAgent. My primary role involves analyzing and aligning conversations to maintain topic relevance, ensuring that all discussions about code commits and reviews stay focused and on track. As a sophisticated component of the AI system, I apply advanced algorithms, including Chain-of-Thought reasoning and optimization techniques, to evaluate and guide conversational flow. I am adept at identifying and correcting topic drifts, ensuring that every conversation adheres to its intended purpose. My capabilities extend to facilitating clear and effective communication between team members, making me an essential asset in streamlining code review processes and enhancing overall team collaboration and decision-making

#### Vulnerabilities

![](res/09_codeagent_vulnerabilities.png)

# 3. Exercise

Using LangChain implement code generation agent.

# 4. References

- [Reflexion: Language Agents with Verbal Reinforcement Learning](https://arxiv.org/abs/2303.11366), [Reflectiong on Reflexion](https://nanothoughts.substack.com/p/reflecting-on-reflexion)
- [The Complete Beginners Guide To Autonomous Agents](https://www.mattprd.com/p/the-complete-beginners-guide-to-autonomous-agents)
- [Autonomous Agents & Agent Simulations](https://blog.langchain.dev/agents-round/)
- [The Anatomy of Autonomy: Why Agents are the next AI Killer App after ChatGPT](https://www.latent.space/p/agents)
- [LLM Powered Autonomous Agents](https://lilianweng.github.io/posts/2023-06-23-agent/)
- [Communicative Agents for Software Development](https://arxiv.org/abs/2307.07924)
- [AutoGen: Enabling Next-Gen LLM Applications via Multi-Agent Conversation](https://arxiv.org/abs/2308.08155)
- [AutoAgents: A Framework for Automatic Agent Generation](https://arxiv.org/abs/2309.17288v2)
- [LLM as OS, Agents as Apps: Envisioning AIOS, Agents and the AIOS-Agent Ecosystem](https://arxiv.org/abs/2312.03815)
- [CodeAgent: Collaborative Agents for Software Engineering](https://arxiv.org/abs/2402.02172)
- [Introducing Devin, the first AI software engineer](https://www.cognition-labs.com/introducing-devin)
- [Large Language Model Evaluation Via Multi AI Agents: Preliminary results](https://arxiv.org/abs/2404.01023)
- [Self-Organized Agents: A LLM Multi-Agent Framework toward Ultra Large-Scale Code Generation and Optimization](https://arxiv.org/abs/2404.02183)
- [Iterative Experience Refinement of Software-Developing Agents](https://arxiv.org/abs/2405.04219)
- [Magentic-One: A Generalist Multi-Agent System for Solving Complex Tasks](https://www.microsoft.com/en-us/research/articles/magentic-one-a-generalist-multi-agent-system-for-solving-complex-tasks/)